In [1]:
import pandas as pd
import vk_api
import requests

In [2]:
def get_id_vk(name):
    id_vk=vk_session.method('utils.resolveScreenName', {'screen_name': name})
    return id_vk['object_id']

In [3]:
def force(f, delay=0.5):
  """При неудачном запросе сделать паузу и попробовать снова"""
  def tmp(*args, **kwargs):
    while True:
      try:
        res = f(*args, **kwargs)
        break
      except KeyError:
        time.sleep(delay)
    return res
  return tmp

In [ ]:
vk_session=vk_api.VkApi('tel', 'parol', app_id=2685278)
vk_session.auth()
vk=vk_session.get_api()

In [5]:
tables = pd.read_csv("data_name.csv")
vk_id=[]
for i in tables.iloc:
    vk_id.append(get_id_vk(i['id']))
tables['vk_id']=vk_id
tables['level']=0

In [6]:
tables

,ФИО,id,vk_id,level
0,Алексеева Екатерина,kate_al02,181400458,0
1,Бирюков Виктор,iktovr,136004593,0
2,Васильев Даниил,da_mil_ka,151143124,0
3,Воронов Кирилл,id248826936,248826936,0
4,Гребенков Дмитрий,irnevoir,172224066,0
5,Ляшун Дмитрий,id356135268,356135268,0
6,Новиков Никита,rapcor,118466021,0
7,Пивницкий Даниэль,slast1_one,625131977,0
8,Уваров Андрей,anonumovich,206240342,0
9,Шубин Григорий,mr.kekess,196085241,0


In [7]:
tables.drop(columns = ['id'],axis = 1, inplace=True) 
tables

,ФИО,vk_id,level
0,Алексеева Екатерина,181400458,0
1,Бирюков Виктор,136004593,0
2,Васильев Даниил,151143124,0
3,Воронов Кирилл,248826936,0
4,Гребенков Дмитрий,172224066,0
5,Ляшун Дмитрий,356135268,0
6,Новиков Никита,118466021,0
7,Пивницкий Даниэль,625131977,0
8,Уваров Андрей,206240342,0
9,Шубин Григорий,196085241,0


In [8]:
@force
def get_friend(vk_id):
    url = f"https://api.vk.com/method/friends.get?user_id={vk_id}&fields=bdate&access_token=vk1.a.B0uObq-7WsrsKwY7tjdp4LgIc64DfyAC09drES0NS63CC5CXIhE-pTWGL_MEBGxK64RKtW6cl2Zw1bfUFi9n_1_GoIM13TI3efOO1l2kbxtpeO1lPLANkVA-zpSVTLlwxDnGSUWMiTUjUM0kix5J4dmxkXE9Ue65WjIxaLhggfIOk98OvLXpzBKDSI-D7lHMiWccFsi97VV2SIoIueN_yQ&v=5.154"
    data = requests.get(url).json()
    return data

In [9]:
level_1={'ФИО':[], 'vk_id':[]}
dict_for_graph={}
for i in tables.iloc:
    friends=[]
    data=get_friend(i['vk_id'])
    if 'error' not in data:
        for j in data['response']['items']:
            name=j['last_name']+' '+j['first_name']
            vk_id=j['id']
            friends.append(vk_id)
            level_1['ФИО'].append(name)
            level_1['vk_id'].append(vk_id)
    dict_for_graph[i['vk_id']]=friends
        

In [10]:
df_level_1 = pd.DataFrame(level_1)
df_level_1['level']=1
df_level_1.drop_duplicates(inplace=True)

In [11]:
df_level_1

,ФИО,vk_id,level
0,Зилов Виктор,4376118,1
1,Гончаров Константин,4572608,1
2,Мугинов Руслан,9751803,1
3,Ii Владислав,20088675,1
4,Алексеев Владислав,82156976,1
...,...,...,...
951,Крестьянников Дмитрий,591996670,1
952,Литковский Никита,596120747,1
953,Pivnitskiy Daniel,625131977,1
954,Korolev Ilya,672453778,1


In [12]:
level_2={'ФИО':[], 'vk_id':[]}
for i in df_level_1.iloc:
    friends=[]
    data=get_friend(i['vk_id'])
    if 'error' not in data:
        for j in data['response']['items']:
            name=j['last_name']+' '+j['first_name']
            vk_id=j['id']
            friends.append(vk_id)
            level_2['ФИО'].append(name)
            level_2['vk_id'].append(vk_id)
    dict_for_graph[i['vk_id']]=friends

In [13]:
df_level_2 = pd.DataFrame(level_2)
df_level_2['level']=2
df_level_2.drop_duplicates(inplace=True)
df_level_2

,ФИО,vk_id,level
0,DELETED,14514,2
1,Иванов Владимир,133501,2
2,Гайсина Наталия,237181,2
3,Zakharova Nadya,508316,2
4,Колодина Анастасия,508941,2
...,...,...,...
167657,Гончаров Александр,331291582,2
167658,Аликин Семён,356193754,2
167659,Чернов Александр,368608228,2
167665,Собкович Юлия,457754097,2


In [14]:
df_people=pd.concat([tables, df_level_1, df_level_2])
df_people.drop_duplicates(inplace=True)

In [15]:
ran=[i for i in range(0, df_people.shape[0])]
df_people['id']=ran
df_people=df_people[['id', 'vk_id', 'ФИО']]
df_people.to_csv('people.csv')

In [16]:
start_pos=[]
target_pos=[]
for k,v in dict_for_graph.items():
    temp=[k]*len(v)
    start_pos.extend(temp)
    target_pos.extend(v)

In [17]:
dict_for_graph_2={'start_pos':start_pos, 'target_pos':target_pos}

In [35]:
df_dict_for_graph=pd.DataFrame(dict_for_graph_2)

In [36]:
df_dict_for_graph.to_csv('data_for_graph.csv')